## Sagemaker Tutorial Series

### Tutorial - 1 Mobile Price Classification using SKLearn Custom Script in Sagemaker

Data Source - https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification?resource=download

### Let's divide the workload
1. Initialize Boto3 SDK and create S3 bucket. 
2. Upload data in Sagemaker Local Storage. 
3. Data Exploration and Understanding.
4. Split the data into Train/Test CSV File. 
5. Upload data into the S3 Bucket.
6. Create Training Script
7. Train script in-side Sagemaker container. 
8. Store Model Artifacts(model.tar.gz) into the S3 Bucket. 
9. Deploy Sagemaker Endpoint(API) for trained model, and test it. 

In [3]:
!pip install scikit-learn

In [4]:
import sklearn # Check Sklearn version
sklearn.__version__

'1.6.1'

In [5]:
!python --version

Python 3.13.1


In [6]:
!pip install -U sagemaker

## 1. Initialize Boto3 SDK and create S3 bucket. 

In [7]:
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'mainbucketrockhight5461' # Mention the created S3 bucket name here
print("Using bucket " + bucket)
# hi
print(f"sagemaker version: {sagemaker.__version__}")

/home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/25/25 18:16:08] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=271018;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=902646;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/murivirg/.config/sagemaker/config.yaml


[03/25/25 18:16:09] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=686938;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=466594;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py#1278\1278]8;;\

Using bucket mainbucketrockhight5461
sagemaker version: 2.242.0


## 3. Data Exploration and Understanding.

In [8]:
df = pd.read_csv("mob_price_classification_train.csv")

In [9]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [10]:
df.shape

(2000, 21)

In [11]:
# ['Low_Risk','High_Risk'],[0,1]
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [12]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [13]:
df.shape

(2000, 21)

In [14]:
# Find the Percentage of Values are missing
df.isnull().mean() * 100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [15]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [16]:
label = features.pop(-1)
label

'price_range'

In [17]:
x = df[features]
y = df[label]

In [18]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [19]:
# {0: 'Low_Risk',1: 'High_Risk'}
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [20]:
x.shape

(2000, 20)

In [21]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [23]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


## 4. Split the data into Train/Test CSV File. 

In [24]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [25]:
print(trainX.shape)
print(testX.shape)

(1700, 21)
(300, 21)


In [26]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,...,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,...,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,...,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,...,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,...,48,1012,959,17,7,6,0,1,0,0


In [27]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [28]:
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

## 5. Upload data into the S3 Bucket.

In [29]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [30]:
# send data to S3. SageMaker will take training data from s3
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [31]:
testpath

's3://mainbucketrockhight5461/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv'

In [32]:
trainpath

's3://mainbucketrockhight5461/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv'

## 6. Create Training Script

In [33]:
%%writefile train.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import sklearn
import pickle  # Replace joblib with pickle
import argparse
import os
import pandas as pd

if __name__ == "__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # Hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    # Removed Joblib version print since joblib is no longer used

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ", label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model.....")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    
    # Change file name to model.pkl and use pickle to save
    model_path = os.path.join(args.model_dir, "model.pkl")
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    print("Model persisted at " + model_path)
    print()

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Overwriting train.py


In [34]:
%%writefile predict.py

import pickle  # Replace joblib with pickle
import os
import pandas as pd
from io import StringIO

def model_fn(model_dir):
    # Load the model using pickle from model.pkl
    with open(os.path.join(model_dir, "model.pkl"), 'rb') as f:
        clf = pickle.load(f)
    return clf

def input_fn(request_body, request_content_type):
    print(request_body)
    print(request_content_type)
    if request_content_type == "text/csv":
        request_body = request_body.strip()
        try:
            df = pd.read_csv(StringIO(request_body), header=None)
            return df
        except Exception as e:
            print(e)
    else:
        return "Please use Content-Type = 'text/csv' and, send the request!!"

def predict_fn(input_data, model):
    if isinstance(input_data, pd.DataFrame):
        prediction = model.predict(input_data)
        print(prediction)
        return prediction
    else:
        return input_data

Overwriting predict.py


In [ ]:
! python train.py --n_estimators 100 \
                   --random_state 0 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \

## 7. save the model.pkl into model.tar.gz

In [ ]:
import tarfile

with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("model.pkl")

### 7.1 we will test the model locally

In [35]:
testX[features][0:2].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0]]

In [ ]:
import pickle
import pandas as pd
from io import StringIO
import numpy as np

# Step 1: Load the model from model.pkl
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

# Step 2: Prepare test data (similar to the endpoint)
# Take the first two rows of testX[features]
test_data = testX[features][0:2].values.tolist()

# Convert to CSV string (mimicking CSVSerializer)
csv_buffer = StringIO()
pd.DataFrame(test_data).to_csv(csv_buffer, header=False, index=False)
csv_data = csv_buffer.getvalue()

# Step 3: Parse the CSV string back to a DataFrame (mimicking input_fn in predict.py)
df_input = pd.read_csv(StringIO(csv_data), header=None)

# Step 4: Make predictions using the loaded model
predictions = model.predict(df_input)

# Step 5: Convert predictions to NumPy array (mimicking NumpyDeserializer)
result = np.array(predictions)

# Step 6: Print the result
print("Predictions:", result)

## 8. Store Model Artifacts(model.tar.gz) into the S3 Bucket. 

In [ ]:
s3 = boto3.client('s3')

# Upload the tar.gz file to S3
s3.upload_file("model.tar.gz", bucket, "models/model.tar.gz")
model_data = f"s3://{bucket}/models/model.tar.gz"

print(f"model data: {model_data}")

## 9. Deploy Sagemaker Endpoint(API) for trained model, and test it. 

In [ ]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(f"framework version: {sklearn.__version__}")
model = SKLearnModel(
    name =  model_name,
    model_data=model_data,
    role=get_execution_role(),
    entry_point="predict.py",
    framework_version="1.2-1",
    dependencies=['requirements.txt']
)

In [ ]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

In [ ]:
testX[features][0:2].values.tolist()

In [ ]:
import pandas as pd
from io import StringIO
from sagemaker.deserializers import NumpyDeserializer
from sagemaker.serializers import CSVSerializer

# Convert testX[features][0:2] to CSV string
test_data = testX[features][0:2].values.tolist()
csv_buffer = StringIO()
pd.DataFrame(test_data).to_csv(csv_buffer, header=False, index=False)
csv_data = csv_buffer.getvalue()

# Set up the predictor with appropriate serializer and deserializer
predictor.serializer = CSVSerializer()
predictor.deserializer = NumpyDeserializer()

# Use predictor.predict with explicit content type
predictor.content_type = "text/csv"  # Set the content type for the request
predictor.accept = "application/x-npy"  # Set the accept type for the response

# Make the prediction
result = predictor.predict(csv_data)
print(result)

## Don't forget to delete the endpoint !

In [ ]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

### Don't forget to Subscribe Machine Learning Hub YouTube Channel. 

In [ ]:
!pip install flask

In [38]:
import sys
import os
import threading
import time
import requests
import pandas as pd
from io import StringIO

# Add docker-folder/src to sys.path to import prediction_script
sys.path.append(os.path.abspath("docker-folder/src"))

# Import the Flask app from prediction_script.py
from prediction_script import app


test_data = pd.DataFrame([
    [1454.0, 1.0, 0.5, 1.0, 1.0, 0.0, 34.0, 0.7, 83.0, 4.0, 3.0, 250.0, 1033.0, 3419.0, 7.0, 5.0, 5.0, 1.0, 1.0, 0.0],
    [1092.0, 1.0, 0.5, 1.0, 10.0, 0.0, 11.0, 0.5, 167.0, 3.0, 14.0, 468.0, 571.0, 737.0, 14.0, 4.0, 11.0, 0.0, 1.0, 0.0]
])


csv_data = test_data.to_csv(header=False, index=False)
print("Sample CSV data for testing:\n", csv_data)

# Define the port (use 8081 to match previous troubleshooting)
PORT = "8081"
LOCAL_URL = f"http://localhost:{PORT}"

# Function to run the Flask app in a separate thread
def run_app():
    # Ensure model.pkl is in the expected path relative to the script
    os.makedirs("/opt/ml/model", exist_ok=True)
    if not os.path.exists("/opt/ml/model/model.pkl"):
        # Copy model.pkl to /opt/ml/model if it’s in the root directory
        if os.path.exists("model.pkl"):
            import shutil
            shutil.copy("model.pkl", "/opt/ml/model/model.pkl")
        else:
            raise FileNotFoundError("model.pkl not found in current directory or /opt/ml/model")
    app.run(host="0.0.0.0", port=PORT, debug=False)

# Start the Flask app in a background thread
print("\nStarting Flask app...")
thread = threading.Thread(target=run_app)
thread.daemon = True  # Allows notebook to stop the thread on interrupt
thread.start()

# Wait for the server to start
time.sleep(2)

# Test /ping endpoint
print("\nTesting /ping endpoint...")
try:
    response = requests.get(f"{LOCAL_URL}/ping", timeout=5)
    response.raise_for_status()
    print("Ping status:", response.status_code)
except requests.RequestException as e:
    print("Ping failed:", e)

# Test /invocations endpoint
print("\nTesting /invocations endpoint...")
headers = {"Content-Type": "text/csv"}
try:
    response = requests.post(f"{LOCAL_URL}/invocations", headers=headers, data=csv_data, timeout=5)
    response.raise_for_status()
    print("Predictions:", response.json())
except requests.RequestException as e:
    print("Invocations failed:", e)
    if "response" in locals():
        print("Response content:", response.text)

# Note: The server will keep running until the kernel is interrupted
print("\nFlask app is running in the background. Interrupt the kernel to stop it.")

Sample CSV data for testing:
 1454.0,1.0,0.5,1.0,1.0,0.0,34.0,0.7,83.0,4.0,3.0,250.0,1033.0,3419.0,7.0,5.0,5.0,1.0,1.0,0.0
1092.0,1.0,0.5,1.0,10.0,0.0,11.0,0.5,167.0,3.0,14.0,468.0,571.0,737.0,14.0,4.0,11.0,0.0,1.0,0.0


Starting Flask app...
 * Serving Flask app 'prediction_script'
 * Debug mode: off


Address already in use
Port 8081 is in use by another program. Either identify and stop that program, or start the server with a different port.



Testing /ping endpoint...


[03/25/25 18:19:23] INFO     127.0.0.1 - - [25/Mar/2025 18:19:23] "GET /ping HTTP/1.1" 200 -        ]8;id=696056;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/werkzeug/_internal.py\_internal.py]8;;\:]8;id=14624;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/werkzeug/_internal.py#97\97]8;;\

Ping status: 200

Testing /invocations endpoint...


[03/25/25 18:19:24] INFO     127.0.0.1 - - [25/Mar/2025 18:19:24] "POST /invocations HTTP/1.1" 200  ]8;id=202970;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/werkzeug/_internal.py\_internal.py]8;;\:]8;id=91807;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/werkzeug/_internal.py#97\97]8;;\
                             -                                                                                     

Predictions: [3, 0]

Flask app is running in the background. Interrupt the kernel to stop it.


In [47]:
import argparse
import numpy
import pandas
import sklearn
import xgboost
import sagemaker
import flask
import gunicorn
import boto3

print(f"argparse: {argparse.__version__}")
print(f"numpy: {numpy.__version__}")
print(f"pandas: {pandas.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"xgboost: {xgboost.__version__}")
print(f"sagemaker: {sagemaker.__version__}")
print(f"flask: {flask.__version__}")
print(f"gunicorn: {gunicorn.__version__}")
print(f"boto3: {boto3.__version__}")

argparse: 1.1
numpy: 1.26.4
pandas: 2.2.3
scikit-learn: 1.6.1
xgboost: 2.1.3
sagemaker: 2.242.0
flask: 1.1.1
gunicorn: 23.0.0
boto3: 1.36.24


/tmp/ipykernel_19743/2204145954.py:17: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print(f"flask: {flask.__version__}")


In [3]:
import requests

# Define the ping endpoint URL
ping_url = "http://localhost:8081/ping"

try:
    # Send GET request with a 5-second timeout
    response = requests.get(ping_url, timeout=5)
    response.raise_for_status()  # Raises an exception for 4xx/5xx status codes
    print("Ping successful, status code:", response.status_code)
except requests.RequestException as e:
    print("Ping failed:", e)

Ping successful, status code: 200


In [4]:
import pandas as pd
import requests
from io import StringIO

# Define test data (same as in your test script)
test_data = pd.DataFrame([
    [1454.0, 1.0, 0.5, 1.0, 1.0, 0.0, 34.0, 0.7, 83.0, 4.0, 3.0, 250.0, 1033.0, 3419.0, 7.0, 5.0, 5.0, 1.0, 1.0, 0.0],
    [1092.0, 1.0, 0.5, 1.0, 10.0, 0.0, 11.0, 0.5, 167.0, 3.0, 14.0, 468.0, 571.0, 737.0, 14.0, 4.0, 11.0, 0.0, 1.0, 0.0]
])

# Convert DataFrame to CSV string without headers or index
csv_data = test_data.to_csv(header=False, index=False)

# Define the invocations endpoint URL
invocations_url = "http://localhost:8081/invocations"

# Set headers for CSV content type
headers = {"Content-Type": "text/csv"}

try:
    # Send POST request with CSV data and a 5-second timeout
    response = requests.post(invocations_url, headers=headers, data=csv_data, timeout=5)
    response.raise_for_status()  # Raises an exception for 4xx/5xx status codes
    predictions = response.json()  # Parse the JSON response
    print("Predictions:", predictions)
except requests.RequestException as e:
    print("Invocations failed:", e)
    if "response" in locals():  # Check if response exists before accessing it
        print("Response content:", response.text)

Predictions: [3, 0]


In [6]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.deserializers import JSONDeserializer
import pandas as pd
import json

# Get the SageMaker execution role (assumes this is run in a SageMaker notebook)
role = "arn:aws:iam::794038231401:role/service-role/SageMaker-ExecutionRole-20250103T203496"

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Specify your ECR image URI (replace with your actual URI)
image_uri = '794038231401.dkr.ecr.us-east-1.amazonaws.com/my-sagemaker-model:latest'

# Create a dummy model artifact (required by SageMaker, even if the model is in the container)
dummy_file = 'dummy.txt'
with open(dummy_file, 'w') as f:
    f.write('dummy')

[03/25/25 21:15:00] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=482102;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=815410;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py#1278\1278]8;;\

In [11]:
import tarfile
with tarfile.open('dummy.tar.gz', 'w:gz') as tar:
    tar.add('dummy.txt')

In [12]:
# Upload the dummy file to S3
def_bucket = sagemaker_session.default_bucket()
def_bucket = sagemaker_session.default_bucket()
print(def_bucket)
model_data = sagemaker_session.upload_data(
    path='model.tar.gz', 
    bucket=def_bucket, 
    key_prefix='model')



sagemaker-us-east-1-794038231401


In [13]:
print(model_data)

s3://sagemaker-us-east-1-794038231401/model/model.tar.gz


In [14]:
# Create the SageMaker model
model = Model(
    image_uri=image_uri,
    model_data=model_data,
    role=role,
    sagemaker_session=sagemaker_session
)

# Optional: Delete the endpoint after testing to avoid costs
# predictor.delete_endpoint()

In [16]:
# Deploy the model to an endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',  # Adjust instance type as needed
    endpoint_name='my-endpoint'   # Replace with a unique endpoint name
)

[03/25/25 21:26:56] INFO     Creating model with name: my-sagemaker-model-2025-03-26-03-26-56-165   ]8;id=174797;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=676166;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name my-endpoint                         ]8;id=6607;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=875459;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py#5937\5937]8;;\

[03/25/25 21:26:57] INFO     Creating endpoint with name my-endpoint                                ]8;id=947862;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=895191;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/sagemaker/session.py#4759\4759]8;;\

-------!

In [18]:
print(type(predictor))

<class 'NoneType'>


In [19]:
import pandas as pd
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.deserializers import JSONDeserializer

# Create predictor object for existing endpoint
predictor = Predictor(endpoint_name='my-endpoint', sagemaker_session=sagemaker.Session())
predictor.deserializer = JSONDeserializer()

# Test data
test_data = pd.DataFrame([
    [1454.0, 1.0, 0.5, 1.0, 1.0, 0.0, 34.0, 0.7, 83.0, 4.0, 3.0, 250.0, 1033.0, 3419.0, 7.0, 5.0, 5.0, 1.0, 1.0, 0.0],
    [1092.0, 1.0, 0.5, 1.0, 10.0, 0.0, 11.0, 0.5, 167.0, 3.0, 14.0, 468.0, 571.0, 737.0, 14.0, 4.0, 11.0, 0.0, 1.0, 0.0]
])
csv_data = test_data.to_csv(header=False, index=False)

# Make prediction
try:
    response = predictor.predict(csv_data, initial_args={'ContentType': 'text/csv'})
    print("Predictions:", response)
except Exception as e:
    print("Prediction failed:", e)

[03/25/25 21:36:21] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=274702;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=16740;file:///home/murivirg/.pyenv/versions/3.13.1/lib/python3.13/site-packages/botocore/credentials.py#1278\1278]8;;\

Predictions: [3, 0]
